# ติดตามการฝึกโมเดลใน Notebook ด้วย MLflow

คุณสามารถใช้ MLflow ใน notebook เพื่อติดตามโมเดลที่คุณฝึก เนื่องจากคุณจะรัน notebook นี้บน Azure Machine Learning compute instance คุณไม่จำเป็นต้องติดตั้ง MLflow เพราะมันถูกติดตั้งและเชื่อมต่อไว้แล้ว

คุณจะเตรียมข้อมูลและฝึกโมเดลเพื่อทำนายโรคเบาหวาน คุณจะใช้ทั้งการบันทึกอัตโนมัติและการบันทึกแบบกำหนดเองเพื่อเรียนรู้วิธีใช้ MLflow ใน notebook

## ก่อนเริ่มต้น

คุณต้องมีแพ็กเกจ **azure-ai-ml** เวอร์ชันล่าสุดเพื่อรันโค้ดใน notebook นี้ รันเซลล์ด้านล่างเพื่อตรวจสอบว่าติดตั้งแล้วหรือยัง

> **หมายเหตุ**:
> หากยังไม่ได้ติดตั้งแพ็กเกจ **azure-ai-ml** ให้รัน `pip install azure-ai-ml` เพื่อติดตั้ง

In [ ]:
pip show azure-ai-ml

## เชื่อมต่อกับ Workspace ของคุณ

หลังจากติดตั้งแพ็กเกจ SDK ที่จำเป็นแล้ว ตอนนี้คุณพร้อมที่จะเชื่อมต่อกับ workspace ของคุณ

ในการเชื่อมต่อกับ workspace เราต้องใช้พารามิเตอร์ระบุตัวตน ได้แก่ subscription ID, ชื่อ resource group และชื่อ workspace เนื่องจากคุณกำลังใช้งาน compute instance ซึ่งจัดการโดย Azure Machine Learning คุณสามารถใช้ค่าเริ่มต้นในการเชื่อมต่อกับ workspace ได้

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

## ตั้งค่า MLflow

เนื่องจากคุณกำลังรัน notebook นี้บน compute instance ใน Azure Machine Learning studio คุณไม่จำเป็นต้องตั้งค่า MLflow

แต่ก็ควรตรวจสอบให้แน่ใจว่าไลบรารีที่จำเป็นถูกติดตั้งแล้ว

> **หมายเหตุ**:
> หากยังไม่ได้ติดตั้งไลบรารี **mlflow** ให้รัน `pip install mlflow` เพื่อติดตั้ง

In [ ]:
pip show mlflow

## เตรียมข้อมูล

คุณจะฝึกโมเดลจำแนกโรคเบาหวาน ข้อมูลสำหรับฝึกเก็บอยู่ในโฟลเดอร์ **data** ชื่อไฟล์ **diabetes.csv**

ก่อนอื่นมาอ่านข้อมูลกันก่อน:

In [ ]:
import pandas as pd

print("Reading data...")
df = pd.read_csv('./data/diabetes.csv')
df.head()

ถัดไป คุณจะแบ่งข้อมูลออกเป็นฟีเจอร์และป้ายกำกับ (Diabetic):

In [ ]:
print("Splitting data...")
X, y = df[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, df['Diabetic'].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

ตอนนี้คุณมีชุดข้อมูลสี่ชุด:

- `X_train`: ชุดข้อมูลสำหรับฝึกที่มีฟีเจอร์
- `X_test`: ชุดข้อมูลสำหรับทดสอบที่มีฟีเจอร์
- `y_train`: ป้ายกำกับสำหรับชุดข้อมูลฝึก
- `y_test`: ป้ายกำกับสำหรับชุดข้อมูลทดสอบ

คุณจะใช้ชุดข้อมูลเหล่านี้เพื่อฝึกและประเมินโมเดล

## สร้าง MLflow Experiment

ตอนนี้คุณพร้อมที่จะฝึกโมเดล machine learning แล้ว ก่อนอื่นคุณจะต้องสร้าง MLflow experiment ด้วยการสร้าง experiment คุณสามารถจัดกลุ่ม run ทั้งหมดไว้ใน experiment เดียวและทำให้หา run ใน studio ได้ง่ายขึ้น

In [ ]:
import mlflow
experiment_name = "mlflow-experiment-diabetes"
mlflow.set_experiment(experiment_name)

## ฝึกและติดตามโมเดล

ในการติดตามโมเดลที่คุณฝึก คุณสามารถใช้ MLflow และเปิดใช้งานการบันทึกอัตโนมัติ เซลล์ต่อไปนี้จะฝึกโมเดลจำแนกโดยใช้ logistic regression คุณจะสังเกตว่าคุณไม่จำเป็นต้องคำนวณเมตริกการประเมินใดๆ เพราะมันถูกสร้างและบันทึกโดย MLflow โดยอัตโนมัติ

In [ ]:
from sklearn.linear_model import LogisticRegression

with mlflow.start_run():
    mlflow.sklearn.autolog()

    model = LogisticRegression(C=1/0.1, solver="liblinear").fit(X_train, y_train)


คุณยังสามารถใช้การบันทึกแบบกำหนดเองกับ MLflow ได้ คุณสามารถเพิ่มการบันทึกแบบกำหนดเองลงในการบันทึกอัตโนมัติ หรือคุณอาจใช้เฉพาะการบันทึกแบบกำหนดเองเท่านั้น

มาฝึกโมเดลอีกสองตัวด้วย scikit-learn เนื่องจากคุณได้รันคำสั่ง `mlflow.sklearn.autolog()` ไว้แล้ว MLflow จะบันทึกโมเดลที่ฝึกด้วย scikit-learn โดยอัตโนมัติ หากต้องการปิดการบันทึกอัตโนมัติ ให้รันเซลล์ต่อไปนี้:

In [ ]:
mlflow.sklearn.autolog(disable=True)

ตอนนี้คุณสามารถฝึกและติดตามโมเดลโดยใช้เฉพาะการบันทึกแบบกำหนดเอง

เมื่อคุณรันเซลล์ต่อไปนี้ คุณจะบันทึกเพียงหนึ่งพารามิเตอร์และหนึ่งเมตริก

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np

with mlflow.start_run():
    model = LogisticRegression(C=1/0.1, solver="liblinear").fit(X_train, y_train)

    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)

    mlflow.log_param("regularization_rate", 0.1)
    mlflow.log_metric("Accuracy", acc)

เหตุผลที่คุณต้องการติดตามโมเดลก็คือเพื่อเปรียบเทียบผลลัพธ์ของโมเดลที่ฝึกด้วยค่าไฮเปอร์พารามิเตอร์ที่แตกต่างกัน

ตอนนี้คุณเพิ่งฝึกโมเดล logistic regression ด้วยอัตราการควบคุม (regularization rate) 0.1 มาฝึกโมเดลอีกตัวหนึ่ง แต่ครั้งนี้ใช้อัตราการควบคุม 0.01 เนื่องจากคุณกำลังติดตามความแม่นยำด้วย คุณสามารถเปรียบเทียบและตัดสินใจว่าอัตราไหนให้ผลลัพธ์โมเดลที่ดีกว่า

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np

with mlflow.start_run():
    model = LogisticRegression(C=1/0.01, solver="liblinear").fit(X_train, y_train)

    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)

    mlflow.log_param("regularization_rate", 0.01)
    mlflow.log_metric("Accuracy", acc)

อีกเหตุผลหนึ่งในการติดตามผลลัพธ์ของโมเดลคือเมื่อคุณกำลังทดสอบตัวประมาณ (estimator) อื่น โมเดลทั้งหมดที่คุณฝึกมาจนถึงตอนนี้ใช้ตัวประมาณ logistic regression

รันเซลล์ต่อไปนี้เพื่อฝึกโมเดลด้วยตัวประมาณ decision tree classifier และดูว่าความแม่นยำสูงกว่า run อื่นๆ หรือไม่

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np

with mlflow.start_run():
    model = DecisionTreeClassifier().fit(X_train, y_train)

    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)

    mlflow.log_param("estimator", "DecisionTreeClassifier")
    mlflow.log_metric("Accuracy", acc)

สุดท้าย มาลองบันทึก artifact กัน artifact สามารถเป็นไฟล์ใดก็ได้ ตอนนี้เราจะลองบันทึก ROC curve เป็นรูปภาพ และบันทึกรูปภาพนั้นเป็น artifact

รันเซลล์ต่อไปนี้เพื่อบันทึกพารามิเตอร์ เมตริก และ artifact

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
import numpy as np

with mlflow.start_run():
    model = DecisionTreeClassifier().fit(X_train, y_train)

    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)

    # plot ROC curve
    y_scores = model.predict_proba(X_test)

    fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
    fig = plt.figure(figsize=(6, 4))
    # Plot the diagonal 50% line
    plt.plot([0, 1], [0, 1], 'k--')
    # Plot the FPR and TPR achieved by our model
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.savefig("ROC-Curve.png")

    mlflow.log_param("estimator", "DecisionTreeClassifier")
    mlflow.log_metric("Accuracy", acc)
    mlflow.log_artifact("ROC-Curve.png")

ตรวจสอบผลลัพธ์ของโมเดลในหน้า Jobs ของ Azure Machine Learning studio

- คุณจะพบพารามิเตอร์ภายใต้ **Params** ในแท็บ **Overview**
- คุณจะพบเมตริกภายใต้ **Metrics** ในแท็บ **Overview** และในแท็บ **Metrics**
- คุณจะพบ artifact ในแท็บ **Outputs + logs**

![ภาพหน้าจอของแท็บ outputs and logs ในหน้า Jobs](./images/output-logs.png)